In [ ]:
import re
import pandas as pd

In [ ]:
# read data
df = pd.read_csv('df_combined.csv')
#pd.set_option('display.max_colwidth', None)
df['title'].head()

In [ ]:
#df[['content']].head(1)

In [ ]:
df = df.fillna('missing')
df.head()

In [ ]:
def clean_text(text):
    '''Cleans content column'''

    # remove journal homepage URLs
    text = re.sub(r'https?://\S+', '', text) 

    # remove ISSN numbers
    text = re.sub(r'ISSN:\s*\(.*?\)', '', text)

    # remove copyright/license info
    text = re.sub(r'©\s*\d{4}.*?license.*', '', text, flags=re.IGNORECASE)

    # remove unwanted "Downloaded from" lines or publication data
    text = re.sub(r'Downloaded from .+? on \w+ \d{2}, \d{4}', '', text)

    # remove any reference to URLs or links in text that aren’t essential
    text = re.sub(r'\bwww\.\S+', '', text) 

    # remove unwanted citations and references (vurder å fjerne dette)
    text = re.sub(r'REFERENCES AND NOTES.*', '', text, flags=re.DOTALL)  # remove references and notes section
    text = re.sub(r'\(\d+\)', '', text)  # remove inline citations 
    text = re.sub(r'\*\*P.*?Wilcoxon.*?\n', '', text)  # remove statistical results

    # normalize spaces and remove unwanted newlines
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
# apply cleaning
df["cleaned_content"] = df["content"].apply(clean_text)

In [ ]:
#pd.set_option('display.max_colwidth', None)
df[['cleaned_content']].head()

In [ ]:
# save cleaned df to csv file
df.to_csv('df_cleaned.csv', index=False)